# Automated formant measurements

This notebook illustrates how to run several formant trackers on a corpus of audio files and collect the results in order to do ensemble analysis. The ensemble technique involves generating formant estimates using several different algorithms, then choosing the best fit.

The X-Ray Microbeam database will be used for the illustration. This database contains audio and associated articulatory data from 48 speakers. Aligned word and phone labels for the audio are available separately.

In this notebook we generate and cache three formant estimates with (1) the inverse filter control method using `ifcformant` and via LPC analysis with the ESPS `formant` command's (2) covariance and (3) autocorrelation modes. 

There are three sources of background information that will make it easier to follow this notebook, and it is recommended that you consult them if you are lost are want more detail.

1. The [Post-processing and data collection notebook](https://github.com/rsprouse/phonlab/blob/master/doc/post-processing_and_data_collection.ipynb) illustrates the post-processing workflow employed in this notebook.
2. The [`dir2df()` notebook](https://github.com/rsprouse/phonlab/blob/master/doc/Retrieving%20filenames%20in%20a%20directory%20tree%20with%20%60dir2df%28%29%60.ipynb) goes into more detail on the `dir2df()` function, which is used in the post-processing notebook.
1. The [X-Ray Microbeam Database repo](https://github.com/rsprouse/xray_microbeam_database) provides the annotations to the X-Ray Microbeam Database and links to the original data files.

## The workflow

The general workflow for processing files is as follows:

1. Read the input data filenames (*.wav) into a DataFrame.
1. Mirror the source directory structure in the destination directory.
1. Merge input and output filenames to find input files that don't have a corresponding output file.
1. Read existing post-processed formant output filenames, if any, into a DataFrame.
1. Run the formant analysis for each input file that is missing an output and cache the results.

The steps will be illustrated separately for each of the formant analysis methods.       

In [1]:
import os, re
import subprocess
import pandas as pd
from phonlab.utils import dir2df

First we define file locations. The name of the directory containing the microbeam database files is stored in `xraydir`, and three additional cache directories names are stored in `ifcdir`, `covdir`, and `acdir`. These cache directories are where output files for each of the analysis methods will be saved.

In [2]:
xraydir = '/media/sf_Downloads/xray_microbeam_database'
ifcdir = '/media/sf_Downloads/xray_formants_ifc'  # For ifcformant output files
covdir = '/media/sf_Downloads/xray_formants_cov'  # For covariance method output files
acdir = '/media/sf_Downloads/xray_formants_ac'    # For autocorrelation method output files

## Load speaker metadata

Before loading input filenames we load speaker metadata for the xray database that is found in the 'speaker_metadata.csv' file. Use the `read_csv()` function to read the 'subject' and 'sex' columns from it. The .csv file uses 'F' and 'M' in the 'sex' column, and the `converters` parameter replaces these values with 'female' and 'male', which are values compatible with `ifcformant`. 

In [3]:
md = pd.read_csv(
    os.path.join(xraydir, 'speaker_metadata.csv'),
    usecols=['subject', 'sex'],
    converters={'sex': lambda x: 'female' if x == 'F' else 'male'}
).rename_axis({'sex': 'spkr'}, axis='columns')
md.head()

,subject,spkr
0,JW05,female
1,JW06,male
2,JW07,male
3,JW08,male
4,JW09,female


Next we add lpc analysis parameters based on speaker type. The values are defined as strings since they will be used for command line arguments rather than for calculations or comparisons.

In [4]:
lpcargs = pd.DataFrame.from_records([
    ('female', '12', '600'),
    ('male',   '14', '500'),
    ('child',  '10', '700')
], columns=['spkr', 'lpc_order', 'nom_f1'])
lpcargs

,spkr,lpc_order,nom_f1
0,female,12,600
1,male,14,500
2,child,10,700


In [5]:
md = md.merge(lpcargs, on='spkr', how='left')
md.head()

,subject,spkr,lpc_order,nom_f1
0,JW05,female,12,600
1,JW06,male,14,500
2,JW07,male,14,500
3,JW08,male,14,500
4,JW09,female,12,600


# Load xray filenames

Load the .wav files in the xray database. For demonstration purposes we select only a subset of .wav files for analysis.

In [6]:
dirpat = '^JW6'
#dirpat = '^JW' # use this instead for all data directories
fnpat = '^tp(?P<task>00\d)\.wav$'
#fnpat = '^t[ap](?P<task>\d+)\.wav$' # use this instead for all .wav files
wavdf = dir2df(
    xraydir,
    dirpat=dirpat,
    fnpat=fnpat,
    addcols=['dirname', 'barename', 'ext']
)
wavdf.head()

,dirname,relpath,fname,barename,ext,task
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005


## Merge with speaker metadata.

The filenames to be analyzed are stored in `wavdf`. Next we merge it with the metadata in `md`. After merging `wavdf` contains rows with target filenames and formant analysis parameters&mdash;`sex`, `lpc_order`, `nom_f1`.

In [7]:
wavdf = wavdf.merge(
    md,
    left_on='relpath',
    right_on='subject',
    how='left'
)
wavdf.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600


# Load existing `ifcformant` output files into a DataFrame

The input filenames are stored in `wavdf`, and the next step is to load any existing output `ifcformant` output files from its cache directory. The first time through there are no cached output files.

In [8]:
ifcdf = dir2df(ifcdir, addcols=['barename', 'dirname', 'ext'])
ifcdf

,dirname,relpath,fname,barename,ext


## Merge with data directory

Merging the dataframes containing input and output dataframes allows us to find input files that lack a corresponding output file. The rows containing `NaN` in one of the columns suffixed with `_ifc` do not yet have an `ifcformant` output.

In [9]:
ifcdf = wavdf.merge(
    ifcdf,
    left_on=['relpath', 'barename'],
    right_on=['relpath', 'barename'],
    suffixes=['', '_ifc'],
    how='left'
)
ifcdf.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_ifc,fname_ifc,ext_ifc
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,NaN,NaN,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,NaN,NaN,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,NaN,NaN,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,NaN,NaN,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,NaN,NaN,NaN


## Find missing formant output files

Next we select the rows that lack an output file by selecting those that have `NaN` values in the `fname_ifc` column with `isnull()`. New columns are added to these rows that the output directory (`dirname_ifc`) and filename (`fname_ifc`). The resulting dataframe has all of the information necessary to run the `ifcformant` command&mdash;speaker type, as well as input and output filenames.

In [10]:
missingifc = ifcdf[ifcdf.fname_ifc.isnull()]
missingifc.dirname_ifc = ifcdir
missingifc.fname_ifc = missingifc.barename + '.ifc'
missingifc.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_ifc,fname_ifc,ext_ifc
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,/media/sf_Downloads/xray_formants_ifc,tp001.ifc,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,/media/sf_Downloads/xray_formants_ifc,tp002.ifc,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,/media/sf_Downloads/xray_formants_ifc,tp003.ifc,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,/media/sf_Downloads/xray_formants_ifc,tp004.ifc,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,/media/sf_Downloads/xray_formants_ifc,tp005.ifc,NaN


## Define `ifcformant` analysis function

The next step is to define an analysis function that uses a row from `missingifc` as its input to run `ifcformant` and save its output to the cache directory. The `run_ifcformant` function uses `subprocess` to execute `ifcformant` on the values contained in one of these rows.

In [11]:
def run_ifcformant(row, errors='raise'):
    '''Perform formant analysis with the ifcformant command.
    
    Parameters
    ----------
    
    row : namedtuple that contains formant analysis parameters
          in fields:
        'dirname' (base pathname to input .wav file),
        'relpath' (relative path to audio file from dirname),
        'fname' (name of .wav file),
        'barename' (name of .wav file without extension)
        'speaker' (ifcformant speaker type, one of 'female',
            'male', 'child')
        'dirname_ifc' (base cache directory name),
        'fname_ifc' (name of output .ifc file),
             
    errors : str (default 'raise')
        How to handle errors if `check_call()` fails. If
        'ignore', print debug statement to STDERR and return the
        ifcformant return code; if 'raise' immediately reraise
        the CalledProcessError.
        
    Returns
    -------
    
    The `ifcformant` return code is returned by this function,
    0 for success or non-zero for errors.
    '''
    try:
        subprocess.check_call([
            "ifcformant",
            "--speaker", row.spkr,
            "--print-header",
            "--output", os.path.join(row.dirname_ifc, row.relpath, row.fname_ifc),
            os.path.join(row.dirname, row.relpath, row.fname)
        ])
    except subprocess.CalledProcessError as e:
        if errors == 'ignore':
            msg = 'Caught error while invoking ifcformant:\n{:}'.format(e)
            sys.stderr.write(msg)
            return e.returncode
        else:
            raise e
    return 0

## Ensure destination directories are created

Before calling `run_ifcformant` we want to ensure that the appropriate output directories in the cache directory are created. We create these by looping over the unique `relpath` values in `missingifc` and concatenating these with the base cache directory name found in `ifcdir`, then calling `os.makedirs()`.

In [12]:
for cdir in missingifc.relpath.unique():
    os.makedirs(os.path.join(ifcdir, cdir), exist_ok=True)

## Run `ifcformant`

Everything is now prepared for running `ifcformant`. Simply loop over the rows in `missingifc` and call `run_ifcformant()`.

In [13]:
for row in missingifc.itertuples():
    run_ifcformant(row)

## Check your work

Assuming no errors are returned by the preceding cell, reloading filenames in the `ifcformant` cache directory should produce new files:

In [14]:
ifcdf = dir2df(ifcdir, addcols=['barename', 'dirname', 'ext'])
ifcdf.head()

,dirname,relpath,fname,barename,ext
0,/media/sf_Downloads/xray_formants_ifc,JW60,tp001.ifc,tp001,.ifc
1,/media/sf_Downloads/xray_formants_ifc,JW60,tp002.ifc,tp002,.ifc
2,/media/sf_Downloads/xray_formants_ifc,JW60,tp003.ifc,tp003,.ifc
3,/media/sf_Downloads/xray_formants_ifc,JW60,tp004.ifc,tp004,.ifc
4,/media/sf_Downloads/xray_formants_ifc,JW60,tp005.ifc,tp005,.ifc


## Define function for covariance and autocorrelation methods

The second and third analysis methods use the ESPS `formant` command with different parameters. A single function can do both, and the covariance or autocorrelation method can be selected with the `lpc_type` parameter. The other analysis parameters, `lpc_order` and `nom_f1`, are already included in `wavdf` and will be passed as part of a dataframe row.

In [15]:
def run_formant(row, lpc_type, errors='raise'):
    '''
    Run ESPS formant command with covariance or autocorrelation settings.
    
    Parameters
    ----------
    
    row : namedtuple that contains formant analysis parameters
          in fields:
        'dirname' (base pathname to input .wav file),
        'relpath' (relative path to audio file from dirname),
        'fname' (name of .wav file),
        'lpc_order' (order of lpc analysis)
        'nom_f1' (nominal value of first formant frequency, in Hz)
        'dirname_out' (base cache directory name)
        
    lpc_type : str ('cov' for covariance or 'ac' for autocorrelation)
             
    errors : str (default 'raise')
        How to handle errors if `check_call()` fails. If
        'ignore', print debug statement to STDERR and return the
        ifcformant return code; if 'raise' immediately reraise
        the CalledProcessError.
        
    Returns
    -------
    
    The `formant` return code is returned by this function,
    0 for success or non-zero for errors.
    '''
    if lpc_type == 'cov':
        wdur = '0.025'
        lpc_opt = '-t1'
    elif lpc_type == 'ac':
        wdur = '0.049'
        lpc_opt = '-t0'
    try:
        subprocess.check_call([
            "formant",
            "-o", row.lpc_order,
            "-N", row.nom_f1,
            lpc_opt,
            "-w", wdur,
            "-O", os.path.join(row.dirname_out, row.relpath),
            os.path.join(row.dirname, row.relpath, row.fname)
        ])
    except subprocess.CalledProcessError as e:
        if errors == 'ignore':
            msg = 'Caught error while invoking formant:\n{:}'.format(e)
            sys.stderr.write(msg)
            return e.returncode
        else:
            raise e
    return 0

## Run covariance analysis

The workflow for the covariance analysis is mostly the same as for `ifcformant`, except the cache directory is different and a different function is called.

In [16]:
# Load cached covariance .fb files
covdf = dir2df(covdir, fnpat='\.fb$', addcols=['barename', 'dirname', 'ext'])
covdf

,dirname,relpath,fname,barename,ext


In [17]:
# Merge with `wavdf`
covdf = wavdf.merge(
    covdf,
    left_on=['relpath', 'barename'],
    right_on=['relpath', 'barename'],
    suffixes=['', '_out'],
    how='left'
)
covdf.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_out,fname_out,ext_out
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,NaN,NaN,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,NaN,NaN,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,NaN,NaN,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,NaN,NaN,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,NaN,NaN,NaN


In [18]:
# Find missing cached covariance files.
missingcov = covdf[covdf.dirname_out.isnull()]
missingcov.dirname_out = covdir
missingcov.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_out,fname_out,ext_out
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,/media/sf_Downloads/xray_formants_cov,NaN,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,/media/sf_Downloads/xray_formants_cov,NaN,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,/media/sf_Downloads/xray_formants_cov,NaN,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,/media/sf_Downloads/xray_formants_cov,NaN,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,/media/sf_Downloads/xray_formants_cov,NaN,NaN


In [19]:
# Ensure output directories are created.
for cdir in missingcov.relpath.unique():
    os.makedirs(os.path.join(covdir, cdir), exist_ok=True)

In [20]:
# Run formant command with covariance method.
for row in missingcov.itertuples():
    run_formant(row, 'cov')

In [21]:
# Check your work.
covdf = dir2df(covdir, fnpat='\.fb$', addcols=['barename', 'dirname', 'ext'])
covdf.head()

,dirname,relpath,fname,barename,ext
0,/media/sf_Downloads/xray_formants_cov,JW60,tp001.fb,tp001,.fb
1,/media/sf_Downloads/xray_formants_cov,JW60,tp002.fb,tp002,.fb
2,/media/sf_Downloads/xray_formants_cov,JW60,tp003.fb,tp003,.fb
3,/media/sf_Downloads/xray_formants_cov,JW60,tp004.fb,tp004,.fb
4,/media/sf_Downloads/xray_formants_cov,JW60,tp005.fb,tp005,.fb


## Run autocorrelation analysis

Running the autocorrelation analysis is nearly the same as running the covariance analysis, with a different cache directory and `lpc_type`.

In [22]:
# Load cached autocorrelation .fb files
acdf = dir2df(acdir, fnpat='\.fb$', addcols=['barename', 'dirname', 'ext'])
acdf

,dirname,relpath,fname,barename,ext


In [23]:
# Merge with `wavdf`
acdf = wavdf.merge(
    acdf,
    left_on=['relpath', 'barename'],
    right_on=['relpath', 'barename'],
    suffixes=['', '_out'],
    how='left'
)
acdf.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_out,fname_out,ext_out
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,NaN,NaN,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,NaN,NaN,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,NaN,NaN,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,NaN,NaN,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,NaN,NaN,NaN


In [24]:
# Find missing cached autocorrelation files.
missingac = acdf[acdf.dirname_out.isnull()]
missingac.dirname_out = acdir
missingac.head()

,dirname,relpath,fname,barename,ext,task,subject,spkr,lpc_order,nom_f1,dirname_out,fname_out,ext_out
0,/media/sf_Downloads/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,female,12,600,/media/sf_Downloads/xray_formants_ac,NaN,NaN
1,/media/sf_Downloads/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002,JW60,female,12,600,/media/sf_Downloads/xray_formants_ac,NaN,NaN
2,/media/sf_Downloads/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003,JW60,female,12,600,/media/sf_Downloads/xray_formants_ac,NaN,NaN
3,/media/sf_Downloads/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004,JW60,female,12,600,/media/sf_Downloads/xray_formants_ac,NaN,NaN
4,/media/sf_Downloads/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005,JW60,female,12,600,/media/sf_Downloads/xray_formants_ac,NaN,NaN


In [25]:
# Ensure output directories are created.
for cdir in missingac.relpath.unique():
    os.makedirs(os.path.join(acdir, cdir), exist_ok=True)

In [26]:
# Run formant command with autocorrelation method.
for row in missingac.itertuples():
    run_formant(row, 'ac')

In [27]:
# Check your work.
acdf = dir2df(acdir, fnpat='\.fb$', addcols=['barename', 'dirname', 'ext'])
acdf.head()

,dirname,relpath,fname,barename,ext
0,/media/sf_Downloads/xray_formants_ac,JW60,tp001.fb,tp001,.fb
1,/media/sf_Downloads/xray_formants_ac,JW60,tp002.fb,tp002,.fb
2,/media/sf_Downloads/xray_formants_ac,JW60,tp003.fb,tp003,.fb
3,/media/sf_Downloads/xray_formants_ac,JW60,tp004.fb,tp004,.fb
4,/media/sf_Downloads/xray_formants_ac,JW60,tp005.fb,tp005,.fb


Junk follows...

In [ ]:
#index=0.10*np.arange(len(d)), 
f = EspsFea('/Users/ronald/Downloads/xray_formants_ac/syba_di.fb')
d = f.get_data()
pd.DataFrame.from_records(d.tolist(), columns=['f1', 'f2', 'f3', 'f4', 'bw1', 'bw2', 'bw3', 'bw4'])

fbdf = fb2df('/Users/ronald/Downloads/xray_formants_ac/syba_di.fb', step=0.010)
fbdf